In [1]:
import matplotlib.pyplot as plt
import numpy as np
import nilmtk
from os.path import expanduser, join

In [2]:
PATH = "~/PhD/experiments/neural_nilm/data_for_BuildSys2015/ground_truth_and_mains"
PATH = expanduser(PATH)

WINDOW_PER_BUILDING = {
    1: ("2014-12-15", "2014-12-22"),
    2: ("2013-10-03 06:16:00", None),
    3: ("2013-04-01 06:15:05", None),
    4: ("2013-09-24 06:15:14", None),
    5: ("2014-09-01", "2014-09-07")
}

APPLIANCES = [
    (['microwave'], (1, 2, 5)),
    (['fridge', 'fridge freezer', 'freezer'], (1, 2, 4, 5)),
    (['dish washer'], (1, 2, 5)),
    (['kettle'], (1, 2, 3, 4, 5)),
    (['washing machine', 'washer dryer'], (1, 2, 5))
]

FILENAME = '/data/mine/vadeec/merged/ukdale.h5'

In [3]:
dataset = nilmtk.DataSet(FILENAME)

In [4]:
def save(filename, meter):
    filename = join(PATH, filename)
    print("  Filename =", filename)
    data = meter.power_series_all_data(sample_period=6)
    data = data.fillna(0)
    np.savetxt(filename, data.round().astype(int).values, delimiter=',', fmt='%d')
    return data

In [5]:
data_per_building = {building_i: {} for building_i in WINDOW_PER_BUILDING.keys()}

In [7]:
dataset.set_window(*WINDOW_PER_BUILDING[5])
save('building_5_microwave.csv', dataset.buildings[5].elec['microwave'])

  Filename = /home/jack/PhD/experiments/neural_nilm/data_for_BuildSys2015/ground_truth_and_mains/building_5_microwave.csv


2014-09-01 00:00:00+01:00    51
2014-09-01 00:00:06+01:00    51
2014-09-01 00:00:12+01:00    51
2014-09-01 00:00:18+01:00    50
2014-09-01 00:00:24+01:00    51
2014-09-01 00:00:30+01:00    50
2014-09-01 00:00:36+01:00    51
2014-09-01 00:00:42+01:00    51
2014-09-01 00:00:48+01:00    50
2014-09-01 00:00:54+01:00    51
2014-09-01 00:01:00+01:00    50
2014-09-01 00:01:06+01:00    51
2014-09-01 00:01:12+01:00    51
2014-09-01 00:01:18+01:00    50
2014-09-01 00:01:24+01:00    50
2014-09-01 00:01:30+01:00    50
2014-09-01 00:01:36+01:00    51
2014-09-01 00:01:42+01:00    51
2014-09-01 00:01:48+01:00    51
2014-09-01 00:01:54+01:00    50
2014-09-01 00:02:00+01:00    50
2014-09-01 00:02:06+01:00    50
2014-09-01 00:02:12+01:00    51
2014-09-01 00:02:18+01:00    51
2014-09-01 00:02:24+01:00    50
2014-09-01 00:02:30+01:00    50
2014-09-01 00:02:36+01:00    50
2014-09-01 00:02:42+01:00    50
2014-09-01 00:02:48+01:00    50
2014-09-01 00:02:54+01:00    50
                             ..
2014-09-

In [27]:
for appliance_list, buildings in APPLIANCES:
    for building_i in buildings:
        window = WINDOW_PER_BUILDING[building_i]
        print("Set window of", window, "for building", building_i)
        dataset.set_window(*window)
        elec = dataset.buildings[building_i].elec
        
        # save mains
        meter = elec.mains()
        filename = "building_{:d}_mains.csv".format(building_i)
        data = save(filename, meter)
        data_per_building[building_i]['mains'] = data
        
        # save appliance data
        for appliance in appliance_list:
            try:
                meter = elec[appliance]
            except KeyError:
                pass
            else:
                print("  Loading", appliance)
                filename = "building_{:d}_{:s}.csv".format(building_i, appliance_list[0].replace(' ', '_'))
                data = save(filename, meter)
                data_per_building[building_i][appliance_list[0]] = data
                

Set window of ('2014-12-15', '2014-12-22') for building 1
  Filename = building_1_mains.csv
  Loading microwave
  Filename = building_1_microwave.csv
Set window of ('2013-10-03 06:16:00', None) for building 2
  Filename = building_2_mains.csv
  Loading microwave
  Filename = building_2_microwave.csv
Set window of ('2013-04-01 06:15:05', None) for building 3
  Filename = building_3_mains.csv
Set window of ('2014-12-15', '2014-12-22') for building 1
  Filename = building_1_mains.csv
  Loading fridge freezer
  Filename = building_1_fridge_freezer.csv
Set window of ('2013-10-03 06:16:00', None) for building 2
  Filename = building_2_mains.csv
  Loading fridge
  Filename = building_2_fridge_freezer.csv
Set window of ('2013-09-24 06:15:14', None) for building 4
  Filename = building_4_mains.csv
  Loading freezer
  Filename = building_4_fridge_freezer.csv
Set window of ('2014-09-01', '2014-09-07') for building 5
  Filename = building_5_mains.csv
  Loading fridge freezer
  Filename = building_

In [28]:
# check data
for building_i, channels in data_per_building.iteritems():
    fig, ax = plt.subplots()
    for name, data in channels.iteritems():
        ax.plot(data, label=name)
    ax.legend()
    ax.set_title(building_i)
    plt.show()